In [12]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-18 14:27:06--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip.2’

dataset_tsmc2014.zi 100%[===================>]  24.36M  29.2MB/s    in 0.8s    

2024-05-18 14:27:07 (29.2 MB/s) - ‘dataset_tsmc2014.zip.2’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
replace dataset_tsmc2014/dataset_TSMC2014_NYC.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [13]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [14]:
df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
print(df)

        User ID                  Venue ID         Venue category ID  \
0           470  49bbd6c0f964a520f4531fe3  4bf58dd8d48988d127951735   
1           979  4a43c0aef964a520c6a61fe3  4bf58dd8d48988d1df941735   
2            69  4c5cc7b485a1e21e00d35711  4bf58dd8d48988d103941735   
3           395  4bc7086715a7ef3bef9878da  4bf58dd8d48988d104941735   
4            87  4cf2c5321d18a143951b5cec  4bf58dd8d48988d1cb941735   
...         ...                       ...                       ...   
227423      688  3fd66200f964a52000e71ee3  4bf58dd8d48988d1e7931735   
227424      560  4bca32ff0687ef3be789dbcc  4bf58dd8d48988d16c941735   
227425      945  50a77716e4b0b5a9492f6f56  4bf58dd8d48988d103941735   
227426      671  4514efe0f964a520e7391fe3  4bf58dd8d48988d11d941735   
227427      942  4a1e0ca0f964a520bf7b1fe3  4bf58dd8d48988d116941735   

        Venue category name   Latitude  Longitude  Timezone  \
0       Arts & Crafts Store  40.719810 -74.002581      -240   
1                    

In [15]:
from datetime import datetime

def str2date(date_str):

    parsed_date = datetime.strptime(date_str, "%a %b %d %H:%M:%S")
    print(parsed_date)

In [16]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [17]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [18]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
#print(venuesdf)


In [21]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))

G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])

'''
for el1 in venuesdict:
for el2 in venuesdict:
distances.append(distance_between_coordinates(venuesdict[el1]['Latitude'],venuesdict[el1]['Longitude'],venuesdict[el2]['Latitude'],venuesdict[el2]['Longitude']))
'''






100%|██████████| 38332/38332 [45:54<00:00, 13.92it/s] 


"\nfor el1 in venuesdict:\nfor el2 in venuesdict:\ndistances.append(distance_between_coordinates(venuesdict[el1]['Latitude'],venuesdict[el1]['Longitude'],venuesdict[el2]['Latitude'],venuesdict[el2]['Longitude']))\n"

In [22]:
import pickle

spatpG = G

pickle.dump(G,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

In [24]:
#nx.draw(G)
print(G)

Graph with 38333 nodes and 15134650 edges


In [ ]:
#spatial graph construction

print(len(venuesdf))
import networkx as nx

spatG = nx.Graph()

d = 1

# Add nodes and edges with features
for _, row1 in venuesdf.iterrows():

    if not spatG.has_node(row1['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

    for _,row2 in venuesdf.iterrows():


        # Add nodes if not already in the graph


        if not spatG.has_node(row2['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

        # Add edges with features

        if distance_between_coordinates(row1['Latitude'],row1['Longitude'],row2['Latitude'],row2['Longitude']) < d:

            spatG.add_edge(row1['Venue ID'],row2['Venue ID'])


print(spatG)


